In [1]:
# Загрузка необходимых модулей для анализа данных и формирования выборки

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import regex as re
import yaml
from transliterate import translit
from datetime import datetime as dt
from sklearn.model_selection import StratifiedGroupKFold

from scipy.stats import spearmanr

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Загрузка сформированной выборки с разметкой

df_gen = pd.read_csv('df_final.csv')
print(df_gen.shape)
df_gen.head()

(65061, 97)


,idinquiry,time_zajavki,tip_organizatsii,data_zajavki,year_mon_zajavki,quarter_zajavki,year_zajavki,inn,time_zagruzki_zajavki,kolichestvo_kompanij_s_analogichnym_direktorom_v_tom_zhe_regione,...,has_product,cnt_otkritija,cnt_zakrytija,cnt_zajavki,m_cnt_otkritija,m_cnt_zakrytija,m_cnt_zajavki,q_cnt_otkritija,q_cnt_zakrytija,q_cnt_zajavki
0,149105,2018-06-18 03:37:00.000000,ЮЛ,2018-06-18,2018-06,2018Q2,2018,BYKKBSYFTS,2018-06-18 03:34:03.993000,2.0,...,NaN,1,0,1,1,0,1,1,0,1
1,149107,2018-06-18 03:49:00.000000,ИП,2018-06-18,2018-06,2018Q2,2018,MIWYERAPTINW,2018-06-18 03:46:46.243000,0.0,...,NaN,1,0,1,1,0,1,1,0,1
2,149109,2018-06-18 04:12:00.000000,ИП,2018-06-18,2018-06,2018Q2,2018,RXDNYNCXKXRF,2018-06-18 04:09:41.117000,0.0,...,NaN,1,0,1,1,0,1,1,0,1
3,149112,2018-06-18 04:56:00.000001,ЮЛ,2018-06-18,2018-06,2018Q2,2018,EGULYSKZPC,2018-06-18 04:53:47.150000,1.0,...,NaN,1,0,1,1,0,1,1,0,1
4,149115,2018-06-18 05:29:00.000000,ЮЛ,2018-06-18,2018-06,2018Q2,2018,MJULWZDIVP,2018-06-18 05:27:15.587000,1.0,...,NaN,1,0,1,1,0,1,1,0,1


### 2. Анализ признаков

In [3]:
check_nans = pd.DataFrame(df_gen.isna().sum()).rename(columns={0:'Nans'}).sort_values(by='Nans',ascending=False)
check_nans['Nans_percent'] = (check_nans.Nans/len(df_gen))*100
print('Размер выборки: ', df_gen.shape)

drop_columns = list(check_nans[check_nans.Nans_percent>90].index)
print('Количество признаков с пропусками >90%: ', len(drop_columns))
check_nans[check_nans.Nans_percent>90]

Размер выборки:  (65061, 97)
Количество признаков с пропусками >90%:  34


,Nans,Nans_percent
_srabotal_priznak_jul_osuschestvljaet_otchislenija_v_pfr_po_sotrudnikam_,65061,100.000000
_razmer_ustavnogo_kapitala_sumkap,65061,100.000000
_rts,65061,100.000000
_vhozhdenie_v_spisok_nedobrosovestnyh_postavschikov_,65061,100.000000
svedenija_ob_uchreditele_uchastnike_rossijskom_juridicheskom_litse,65061,100.000000
svedenija_ob_uchreditele_uchastnike_inostrannom_juridicheskom_litse,65061,100.000000
nalogovaja_nagruzka,65061,100.000000
svedenija_ob_uchreditele_uchastnike_fizicheskom_litse,65061,100.000000
svedenija_ob_uchreditele_uchastnike_rossijskoj_federatsii_sub_ekte_rossijskoj_federatsii_munitsipal_nom_obrazovanii,65061,100.000000
_kodopf,65061,100.000000


In [4]:
# Удаление из выборки факторов с пропусками более 90%

df = df_gen.drop(columns=drop_columns)
print('Shape after drop factors with Nans > 90 percent: ', df.shape)

Shape after drop factors with Nans > 90 percent:  (65061, 63)


In [5]:
df.head()

,idinquiry,time_zajavki,tip_organizatsii,data_zajavki,year_mon_zajavki,quarter_zajavki,year_zajavki,inn,time_zagruzki_zajavki,kolichestvo_kompanij_s_analogichnym_direktorom_v_tom_zhe_regione,...,target,cnt_otkritija,cnt_zakrytija,cnt_zajavki,m_cnt_otkritija,m_cnt_zakrytija,m_cnt_zajavki,q_cnt_otkritija,q_cnt_zakrytija,q_cnt_zajavki
0,149105,2018-06-18 03:37:00.000000,ЮЛ,2018-06-18,2018-06,2018Q2,2018,BYKKBSYFTS,2018-06-18 03:34:03.993000,2.0,...,0,1,0,1,1,0,1,1,0,1
1,149107,2018-06-18 03:49:00.000000,ИП,2018-06-18,2018-06,2018Q2,2018,MIWYERAPTINW,2018-06-18 03:46:46.243000,0.0,...,0,1,0,1,1,0,1,1,0,1
2,149109,2018-06-18 04:12:00.000000,ИП,2018-06-18,2018-06,2018Q2,2018,RXDNYNCXKXRF,2018-06-18 04:09:41.117000,0.0,...,0,1,0,1,1,0,1,1,0,1
3,149112,2018-06-18 04:56:00.000001,ЮЛ,2018-06-18,2018-06,2018Q2,2018,EGULYSKZPC,2018-06-18 04:53:47.150000,1.0,...,0,1,0,1,1,0,1,1,0,1
4,149115,2018-06-18 05:29:00.000000,ЮЛ,2018-06-18,2018-06,2018Q2,2018,MJULWZDIVP,2018-06-18 05:27:15.587000,1.0,...,1,1,0,1,1,0,1,1,0,1


In [6]:
df.columns.tolist()

['idinquiry',
 'time_zajavki',
 'tip_organizatsii',
 'data_zajavki',
 'year_mon_zajavki',
 'quarter_zajavki',
 'year_zajavki',
 'inn',
 'time_zagruzki_zajavki',
 'kolichestvo_kompanij_s_analogichnym_direktorom_v_tom_zhe_regione',
 'kompanija_vhodit_v_juridicheskie_litsa_v_sostav_ispolnitel_nyh_organov_kotoryh_vhodjat_diskvalifitsirovannye_litsa',
 'kolichestvo_kompanij_zaregistrirovannyh_na_adrese_registratsii_organizatsii_po_dannym_sajta_fns',
 'kolichestvo_vidov_dejatel_nosti_u_klienta',
 'kod_osnovnogo_okved',
 'sub_ekt_mestonahozhdenija',
 'razmer_ustavnogo_kapital_jul',
 'chistaja_pribyl_ili_ubytok_kompanii',
 'otchetnyj_period_god_',
 'passivy_vsego',
 'data_nachala_polnomochij_rukovoditelja',
 'kolichestvo_souchreditelej',
 'daty_vnesenija_souchreditelej',
 'data_registratsii',
 'kolichestvo_filialov',
 'chislennost_kompanii',
 'vyruchka_kompanii_mln_rub_',
 'summa_naloga',
 'opf_organizatsii',
 'kredity',
 '_v_uchrediteljah_uchastnikah_aktsionerah_klienta_uchastie_gosudarstva_b

In [7]:
# Создание бинарных признаков (больше 1 заявки, открытия или закрытия счета или нет)

df['more_one_open'] = np.where(df.cnt_otkritija>1,1,0).astype(object)
df['more_one_close'] = np.where(df.cnt_zakrytija>1,1,0).astype(object)
df['more_one_req'] = np.where(df.cnt_zajavki>1,1,0).astype(object)
df['more_one_open_m'] = np.where(df.m_cnt_otkritija>1,1,0).astype(object)
df['more_one_close_m'] = np.where(df.m_cnt_zakrytija>1,1,0).astype(object)
df['more_one_req_m'] = np.where(df.m_cnt_zajavki>1,1,0).astype(object)
df['more_one_open_q'] = np.where(df.q_cnt_otkritija>1,1,0).astype(object)
df['more_one_close_q'] = np.where(df.q_cnt_zakrytija>1,1,0).astype(object)
df['more_one_req_q'] = np.where(df.q_cnt_zajavki>1,1,0).astype(object)

df['diff_req_open'] = (df.cnt_zajavki-df.cnt_otkritija)

In [8]:
# Составляем список вспомогательных фичей, для исключения из процесса обучения

drop_cols = ['idinquiry',
             'time_zajavki',
             'data_zajavki',
             'year_mon_zajavki',
             'quarter_zajavki',
             'year_zajavki',
             'inn',
             'time_zagruzki_zajavki',
             'data_zagruzki_zajavki',
             'year_mon_zagruzki_zajavki',
             'data_otkrytija',
             'data_zakrytija',
             'year_mon_open',
             'quarter_open',
             'data_blokirovki',
             'year_mon_block',
             'target',
             'cnt_otkritija',
             'cnt_zakrytija',
             'cnt_zajavki',
             'm_cnt_otkritija',
             'm_cnt_zakrytija',
             'm_cnt_zajavki',
             'q_cnt_otkritija',
             'q_cnt_zakrytija',
             'q_cnt_zajavki']

target_col = 'target'

In [9]:
# Количество признаков для дальнейшего анализа

features = list(set(df.columns.to_list())-set(drop_cols))
print('all: ', len(features))

# Формирование первоначальных категориальный признаков

cat_cols = list(df[features].select_dtypes(include=[object]))
print('cat: ',len(cat_cols))

# Формирование первоначальных количественных признаков

cont_cols = list(set(features)-set(cat_cols))
print('cont: ', len(cont_cols))

all:  47
cat:  30
cont:  17


In [10]:
# Проверка категориальных признаков на их уникальные значения

features_uniq = pd.DataFrame(df[cat_cols].nunique()).\
rename(columns={0:'cnt'}).sort_values(by='cnt',ascending=False)
features_uniq

,cnt
data_nachala_polnomochij_rukovoditelja,11636
istorija_smeny_soouchreditelej,11493
daty_vnesenija_souchreditelej,11091
data_registratsii,5332
vyruchka_kompanii_mln_rub_,2210
kod_osnovnogo_okved,1542
sub_ekt_mestonahozhdenija,675
opf_organizatsii,76
chislennost_kompanii,11
indeks_finansovogo_riska_opisanie,4


In [11]:
df[['data_nachala_polnomochij_rukovoditelja','istorija_smeny_soouchreditelej','daty_vnesenija_souchreditelej','data_registratsii']]

,data_nachala_polnomochij_rukovoditelja,istorija_smeny_soouchreditelej,daty_vnesenija_souchreditelej,data_registratsii
0,08.06.2018,ЮЛ: 08.06.2018 0:00:00,08.06.2018,08.06.2018
1,NaN,NaN,NaN,24.05.2012
2,NaN,NaN,NaN,04.06.2018
3,14.06.2018,ЮЛ: 14.06.2018 0:00:00,14.06.2018,14.06.2018
4,19.04.2018,ЮЛ: 19.04.2018 0:00:00,19.04.2018,19.04.2018
...,...,...,...,...
65056,NaN,NaN,NaN,23.12.2002
65057,28.08.2018,ЮЛ: 28.08.2018 0:00:00,28.08.2018,28.08.2018
65058,NaN,NaN,NaN,11.05.2005
65059,02.10.2018;04.10.2016,ФЛ: 04.10.2016 0:00:00,04.10.2016,04.10.2016


In [12]:
df['data_nachala_polnomochij_rukovoditelja'] = df.data_nachala_polnomochij_rukovoditelja.replace(np.nan,'NaN')

df['kolichestvo_change_rukovoditelja'] = df['data_nachala_polnomochij_rukovoditelja'].\
apply(lambda x: len(str(x).split(';')) if x!='NaN' else 0)

In [13]:
# Пока что даты смены/начала/внесения соучередителей и руководителей уберем из списка факторов.

add_drop_cols = ['data_nachala_polnomochij_rukovoditelja',
             'istorija_smeny_soouchreditelej',
             'daty_vnesenija_souchreditelej']

df = df.drop(columns=add_drop_cols)
print(df.shape)
df.head()

(65061, 71)


,idinquiry,time_zajavki,tip_organizatsii,data_zajavki,year_mon_zajavki,quarter_zajavki,year_zajavki,inn,time_zagruzki_zajavki,kolichestvo_kompanij_s_analogichnym_direktorom_v_tom_zhe_regione,...,more_one_close,more_one_req,more_one_open_m,more_one_close_m,more_one_req_m,more_one_open_q,more_one_close_q,more_one_req_q,diff_req_open,kolichestvo_change_rukovoditelja
0,149105,2018-06-18 03:37:00.000000,ЮЛ,2018-06-18,2018-06,2018Q2,2018,BYKKBSYFTS,2018-06-18 03:34:03.993000,2.0,...,0,0,0,0,0,0,0,0,0,1
1,149107,2018-06-18 03:49:00.000000,ИП,2018-06-18,2018-06,2018Q2,2018,MIWYERAPTINW,2018-06-18 03:46:46.243000,0.0,...,0,0,0,0,0,0,0,0,0,0
2,149109,2018-06-18 04:12:00.000000,ИП,2018-06-18,2018-06,2018Q2,2018,RXDNYNCXKXRF,2018-06-18 04:09:41.117000,0.0,...,0,0,0,0,0,0,0,0,0,0
3,149112,2018-06-18 04:56:00.000001,ЮЛ,2018-06-18,2018-06,2018Q2,2018,EGULYSKZPC,2018-06-18 04:53:47.150000,1.0,...,0,0,0,0,0,0,0,0,0,1
4,149115,2018-06-18 05:29:00.000000,ЮЛ,2018-06-18,2018-06,2018Q2,2018,MJULWZDIVP,2018-06-18 05:27:15.587000,1.0,...,0,0,0,0,0,0,0,0,0,1


In [14]:
# Обновляем все признаки и отдельно категориальные

features = list(set(df.columns.to_list())-set(drop_cols))

cat_cols = list(df[features].select_dtypes(include=[object]))
del_cat_cols = ['vyruchka_kompanii_mln_rub_']

cat_cols_new = list(set(cat_cols) - set(del_cat_cols))
len(cat_cols_new)

26

In [15]:
# Дата регистрации возьмем только год-месяц
# Приводим признак выручки в нужный формат рублей
# Исправляем опечатки в уставном капитале, так как есть наблюдения у которых 10 руб, хотя минимальный по закону 10 тыс руб

df['data_registratsii'] = pd.to_datetime(df['data_registratsii']).dt.strftime('%Y-%m')

df['vyruchka_kompanii_mln_rub_'] = df['vyruchka_kompanii_mln_rub_'].apply(lambda x: str(x).replace(',','.')).astype(float)*1000000

df['razmer_ustavnogo_kapital_jul'] = df['razmer_ustavnogo_kapital_jul'].apply(lambda x: x*1000 if x<10000 else x)

In [16]:
# Проверка количественных признаков на их уникальные значения

features_uniq = pd.DataFrame(df[list(set(features)-set(cat_cols_new))].nunique()).\
rename(columns={0:'cnt'}).sort_values(by='cnt',ascending=False)
features_uniq

,cnt
passivy_vsego,12730
chistaja_pribyl_ili_ubytok_kompanii,10645
summa_naloga,6040
vyruchka_kompanii_mln_rub_,2210
razmer_ustavnogo_kapital_jul,955
kolichestvo_vidov_dejatel_nosti_u_klienta,243
_kolichestvo_kompanij_s_analogichnym_direktorom,147
kolichestvo_kompanij_zaregistrirovannyh_na_adrese_registratsii_organizatsii_po_dannym_sajta_fns,144
_massovyj_kontaktnyj_telefon_v_zajavke_ukazan_telefon_na_kotoryj_zaregistrirovano_bolee_h_kompanij_krome_kompanij_agentov_,120
indeks_finansovogo_riska,98


In [17]:
# Добавляем в список новые категориальные признаки, обновляем список количественных признаков

add_cat = ['_massovyj_kontaktnyj_telefon_v_zajavke_ukazan_telefon_na_kotoryj_zaregistrirovano_bolee_h_kompanij_krome_kompanij_agentov_',
          'indeks_finansovogo_riska',
          'indeks_dolzhnoj_osmotritel_nosti',
          'otchetnyj_period_god_',
          'kredity']

cat_cols_final = list(set(cat_cols_new + add_cat))

cont_cols_final = list(set(features)-set(cat_cols_final))

In [18]:
# Количество категориальных и количественных признаков

print('Cnt of categ features: ',len(cat_cols_final))
print('Cnt of cont features: ', len(cont_cols_final))

Cnt of categ features:  31
Cnt of cont features:  14


In [19]:
# Проверка, что никакой признак не потерял

print(len(features),':', len(cont_cols_final+cat_cols_final))

45 : 45


In [20]:
df[['vyruchka_kompanii_mln_rub_','chistaja_pribyl_ili_ubytok_kompanii','passivy_vsego','summa_naloga','razmer_ustavnogo_kapital_jul']].dropna()

,vyruchka_kompanii_mln_rub_,chistaja_pribyl_ili_ubytok_kompanii,passivy_vsego,summa_naloga,razmer_ustavnogo_kapital_jul
32,41600000.0,176700.0,36899520.0,480800.0,10000.0
39,105900000.0,258960.0,85412880.0,110400.0,20000.0
61,261600000.0,6795800.0,124208700.0,1405040.0,10000.0
64,1400000.0,26390.0,7686900.0,-14840.0,10000.0
76,24000000.0,2458820.0,14328030.0,801430.0,250000.0
...,...,...,...,...,...
65027,88000000.0,1295720.0,20084820.0,262350.0,10000.0
65032,364100000.0,3591560.0,82647630.0,795970.0,100000.0
65041,3900000.0,780560.0,7366480.0,122000.0,10000.0
65045,1600000.0,53410.0,1410400.0,12880.0,10000.0


In [21]:
# Создание финансовых признаков ROA (рентабельность активов), ROS (рентабельность продаж),
# и еще разные отношения

df['roa'] = ((df['chistaja_pribyl_ili_ubytok_kompanii']/df['passivy_vsego'])*100).replace([np.inf, -np.inf], 0)
df['ros'] = ((df['chistaja_pribyl_ili_ubytok_kompanii']/df['vyruchka_kompanii_mln_rub_'])*100).replace([np.inf, -np.inf], 0)
df['tax_profit'] = ((df['summa_naloga']/df['chistaja_pribyl_ili_ubytok_kompanii'])*100).replace([np.inf, -np.inf], 0)

add_cont_cols = ['roa', 'ros','tax_profit']

In [22]:
# Обновляем количественные факторы

cont_cols_final = list(set(cont_cols_final + add_cont_cols))
len(cont_cols_final)

17

In [23]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans

def generation_kmeans(X,num_col,k):
    
    X = X[num_col].replace([np.inf, -np.inf], np.nan)
    
    num_col = num_col
    
    # пайплайн для численных признаков
    num_pipe = Pipeline([('imputer', SimpleImputer(strategy='median')),
                         ('scaler', StandardScaler())])
    # соединяем пайплайны вместе
    transformer = ColumnTransformer(
                               transformers=[('num', num_pipe, num_col)], 
                                             remainder='passthrough')
    kmeans = Pipeline([('transformer', transformer),
                               ('k_means', KMeans(n_clusters=k, random_state=42))])

    kmeans.fit(X)

    return kmeans.predict(X)

In [24]:
pd.DataFrame(df[cont_cols_final].nunique()).sort_values(by=0,ascending=False).index.tolist()

['passivy_vsego',
 'roa',
 'ros',
 'chistaja_pribyl_ili_ubytok_kompanii',
 'tax_profit',
 'summa_naloga',
 'vyruchka_kompanii_mln_rub_',
 'razmer_ustavnogo_kapital_jul',
 'kolichestvo_vidov_dejatel_nosti_u_klienta',
 '_kolichestvo_kompanij_s_analogichnym_direktorom',
 'kolichestvo_kompanij_zaregistrirovannyh_na_adrese_registratsii_organizatsii_po_dannym_sajta_fns',
 'kolichestvo_kompanij_s_analogichnym_direktorom_v_tom_zhe_regione',
 '_kolichestvo_kompanij_s_analogichnym_direktorom_v_tom_zhe_regione',
 'kolichestvo_souchreditelej',
 'kolichestvo_change_rukovoditelja',
 'diff_req_open',
 'kolichestvo_filialov']

In [25]:
cont_cols_for_kmeans = ['passivy_vsego',
 'roa',
 'ros',
 'chistaja_pribyl_ili_ubytok_kompanii',
 'tax_profit',
 'summa_naloga',
 'vyruchka_kompanii_mln_rub_',
 'razmer_ustavnogo_kapital_jul',
 'kolichestvo_vidov_dejatel_nosti_u_klienta',
 '_kolichestvo_kompanij_s_analogichnym_direktorom',
 'kolichestvo_kompanij_zaregistrirovannyh_na_adrese_registratsii_organizatsii_po_dannym_sajta_fns',
 'kolichestvo_kompanij_s_analogichnym_direktorom_v_tom_zhe_regione',
 '_kolichestvo_kompanij_s_analogichnym_direktorom_v_tom_zhe_regione',
 'kolichestvo_souchreditelej',
 'kolichestvo_change_rukovoditelja',
 'diff_req_open',
 'kolichestvo_filialov']

In [26]:
for i in range(2,11):
    df['kmeans_'+str(i)] = generation_kmeans(df,cont_cols_for_kmeans,i)
    cat_cols_final.append('kmeans_'+str(i))

In [27]:
pd.DataFrame(df[cat_cols_final].nunique()).sort_values(by=0, ascending=False)

,0
kod_osnovnogo_okved,1542
sub_ekt_mestonahozhdenija,675
data_registratsii,373
_massovyj_kontaktnyj_telefon_v_zajavke_ukazan_telefon_na_kotoryj_zaregistrirovano_bolee_h_kompanij_krome_kompanij_agentov_,120
indeks_dolzhnoj_osmotritel_nosti,98
indeks_finansovogo_riska,98
opf_organizatsii,76
otchetnyj_period_god_,19
chislennost_kompanii,11
kmeans_10,10


In [28]:
pd.DataFrame(df[cat_cols_final].nunique()).sort_values(by=0, ascending=False).index.tolist()

['kod_osnovnogo_okved',
 'sub_ekt_mestonahozhdenija',
 'data_registratsii',
 '_massovyj_kontaktnyj_telefon_v_zajavke_ukazan_telefon_na_kotoryj_zaregistrirovano_bolee_h_kompanij_krome_kompanij_agentov_',
 'indeks_dolzhnoj_osmotritel_nosti',
 'indeks_finansovogo_riska',
 'opf_organizatsii',
 'otchetnyj_period_god_',
 'chislennost_kompanii',
 'kmeans_10',
 'kmeans_9',
 'kmeans_8',
 'kmeans_7',
 'kmeans_6',
 'kmeans_5',
 'kmeans_4',
 'indeks_dolzhnoj_osmotritel_nosti_opisanie',
 'indeks_finansovogo_riska_opisanie',
 'kmeans_3',
 'more_one_req_q',
 'more_one_req_m',
 'kmeans_2',
 'more_one_req',
 'tip_organizatsii',
 'more_one_open',
 'more_one_open_q',
 '_v_uchrediteljah_uchastnikah_aktsionerah_klienta_uchastie_gosudarstva_bolee_',
 'more_one_close',
 'more_one_close_m',
 'more_one_open_m',
 'kredity',
 '_organizatsija_pervoe_litso_uchrediteli_vhodit_v_spisok_diskvalifitsirovannyh_',
 'more_one_close_q',
 '_perechen_oao_po_rasporjazheniju_pravitel_stva_r',
 '_reestr_oboronno_promyshlennogo

In [29]:
features_before_corr = df.drop(columns=drop_cols).columns.tolist()
len(features_before_corr), len(cat_cols_final), len(cont_cols_final)

(57, 40, 17)

In [30]:
features_to_save_before = {'features_before_corr':features_before_corr,
                   'cont_features_before_corr':cont_cols_final,
                   'cat_features_before_corr':cat_cols_final}

with open('features_01.yaml', 'w') as f:
    yaml.dump(features_to_save_before, f)

In [31]:
# Кодируем категориальные фичи, для расчета корреляции

df_corr_analysis = df.copy()

cat_cols_for_corr = ['kmeans_10',
 'kmeans_9',
 'kmeans_8',
 'kmeans_7',
 'kmeans_6',
 'kmeans_5',
 'kmeans_4',
 'indeks_finansovogo_riska_opisanie',
 'indeks_dolzhnoj_osmotritel_nosti_opisanie',
 'kmeans_3',
 'kmeans_2',
 'more_one_close_m',
 'more_one_close',
 'more_one_req_q',
 'more_one_close_q',
 'more_one_open',
 'more_one_req',
 'tip_organizatsii',
 'more_one_open_m',
 '_v_uchrediteljah_uchastnikah_aktsionerah_klienta_uchastie_gosudarstva_bolee_',
 'more_one_open_q',
 '_organizatsija_pervoe_litso_uchrediteli_vhodit_v_spisok_diskvalifitsirovannyh_',
 'more_one_req_m',
 'kredity',
 'kompanija_vhodit_v_juridicheskie_litsa_v_sostav_ispolnitel_nyh_organov_kotoryh_vhodjat_diskvalifitsirovannye_litsa',
 '_jur_litsa_ne_predostavljajuschie_nalogovuju_otchetnost_bolee_goda',
 '_jur_litsa_imejuschie_zadolzhennost_po_uplate_nalogov',
 '_perechen_oao_po_rasporjazheniju_pravitel_stva_r',
 '_reestr_oboronno_promyshlennogo_kompleksa',
 '_perechen_fgup_imejuschih_suschestvennoe_znachenie_',
 '_reestr_operatorov_osuschestvljajuschih_obrabotku_personal_nyh_dannyh']

from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()

for i in cat_cols_for_corr:
    df_corr_analysis[i] = enc.fit_transform(df_corr_analysis[i])
    

In [32]:
# Расчитаем корреляцию Спирмена между количественными признаками, создадим список где корреляция > 90%

corr_matrix = df_corr_analysis[cont_cols_final+cat_cols_for_corr].corr(method='spearman').abs()
print('Корреляции посчитаны')
high_corr_var = np.where(corr_matrix>0.9)
high_corr_var = [(corr_matrix.columns[x], corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]
print('Пары признаков с корреляцией: ', high_corr_var)
corr_cols_to_drop = list(set(x[1] for x in high_corr_var))
print('На удаление: ', corr_cols_to_drop)

Корреляции посчитаны
Пары признаков с корреляцией:  [('_kolichestvo_kompanij_s_analogichnym_direktorom_v_tom_zhe_regione', 'kolichestvo_kompanij_s_analogichnym_direktorom_v_tom_zhe_regione'), ('_kolichestvo_kompanij_s_analogichnym_direktorom_v_tom_zhe_regione', 'kmeans_6'), ('_kolichestvo_kompanij_s_analogichnym_direktorom_v_tom_zhe_regione', 'tip_organizatsii'), ('diff_req_open', 'more_one_req'), ('kolichestvo_change_rukovoditelja', 'kmeans_7'), ('kolichestvo_change_rukovoditelja', 'kmeans_6'), ('kolichestvo_change_rukovoditelja', 'indeks_finansovogo_riska_opisanie'), ('kolichestvo_change_rukovoditelja', 'indeks_dolzhnoj_osmotritel_nosti_opisanie'), ('kolichestvo_change_rukovoditelja', 'tip_organizatsii'), ('kolichestvo_kompanij_s_analogichnym_direktorom_v_tom_zhe_regione', 'kmeans_6'), ('kolichestvo_kompanij_s_analogichnym_direktorom_v_tom_zhe_regione', 'tip_organizatsii'), ('kmeans_9', 'kmeans_8'), ('kmeans_9', 'kmeans_4'), ('kmeans_9', 'kmeans_3'), ('kmeans_7', 'kmeans_6'), ('kmean

In [33]:
corr_cols_to_drop

['_jur_litsa_ne_predostavljajuschie_nalogovuju_otchetnost_bolee_goda',
 'kmeans_3',
 'kmeans_8',
 'kmeans_6',
 'tip_organizatsii',
 'more_one_req',
 'kmeans_4',
 '_reestr_oboronno_promyshlennogo_kompleksa',
 '_perechen_oao_po_rasporjazheniju_pravitel_stva_r',
 '_reestr_operatorov_osuschestvljajuschih_obrabotku_personal_nyh_dannyh',
 'indeks_dolzhnoj_osmotritel_nosti_opisanie',
 '_perechen_fgup_imejuschih_suschestvennoe_znachenie_',
 'kmeans_7',
 '_jur_litsa_imejuschie_zadolzhennost_po_uplate_nalogov',
 'more_one_req_m',
 'kolichestvo_kompanij_s_analogichnym_direktorom_v_tom_zhe_regione',
 'indeks_finansovogo_riska_opisanie']

In [34]:
df_features = df.drop(columns=corr_cols_to_drop).copy()
print(df.shape)
print(df_features.shape)
df_features.head()

(65061, 83)
(65061, 66)


,idinquiry,time_zajavki,data_zajavki,year_mon_zajavki,quarter_zajavki,year_zajavki,inn,time_zagruzki_zajavki,kompanija_vhodit_v_juridicheskie_litsa_v_sostav_ispolnitel_nyh_organov_kotoryh_vhodjat_diskvalifitsirovannye_litsa,kolichestvo_kompanij_zaregistrirovannyh_na_adrese_registratsii_organizatsii_po_dannym_sajta_fns,...,more_one_req_q,diff_req_open,kolichestvo_change_rukovoditelja,roa,ros,tax_profit,kmeans_2,kmeans_5,kmeans_9,kmeans_10
0,149105,2018-06-18 03:37:00.000000,2018-06-18,2018-06,2018Q2,2018,BYKKBSYFTS,2018-06-18 03:34:03.993000,Нет,3.0,...,0,0,1,NaN,NaN,NaN,0,1,3,1
1,149107,2018-06-18 03:49:00.000000,2018-06-18,2018-06,2018Q2,2018,MIWYERAPTINW,2018-06-18 03:46:46.243000,Нет,NaN,...,0,0,0,NaN,NaN,NaN,0,0,0,5
2,149109,2018-06-18 04:12:00.000000,2018-06-18,2018-06,2018Q2,2018,RXDNYNCXKXRF,2018-06-18 04:09:41.117000,Нет,NaN,...,0,0,0,NaN,NaN,NaN,0,0,0,5
3,149112,2018-06-18 04:56:00.000001,2018-06-18,2018-06,2018Q2,2018,EGULYSKZPC,2018-06-18 04:53:47.150000,Нет,2.0,...,0,0,1,NaN,NaN,NaN,0,0,0,5
4,149115,2018-06-18 05:29:00.000000,2018-06-18,2018-06,2018Q2,2018,MJULWZDIVP,2018-06-18 05:27:15.587000,Нет,2.0,...,0,0,1,NaN,NaN,NaN,0,0,3,1


In [35]:
print('Features after corr: ')
features_after_corr = list(set(cont_cols_final+cat_cols_final)-set(corr_cols_to_drop))
len(features_after_corr)

Features after corr: 


40

In [36]:
print('Cont features: ')
print()
cont_cols_after_corr = list(set(cont_cols_final)-set(corr_cols_to_drop))
len(cont_cols_after_corr), sorted(cont_cols_after_corr)

Cont features: 



(16,
 ['_kolichestvo_kompanij_s_analogichnym_direktorom',
  '_kolichestvo_kompanij_s_analogichnym_direktorom_v_tom_zhe_regione',
  'chistaja_pribyl_ili_ubytok_kompanii',
  'diff_req_open',
  'kolichestvo_change_rukovoditelja',
  'kolichestvo_filialov',
  'kolichestvo_kompanij_zaregistrirovannyh_na_adrese_registratsii_organizatsii_po_dannym_sajta_fns',
  'kolichestvo_souchreditelej',
  'kolichestvo_vidov_dejatel_nosti_u_klienta',
  'passivy_vsego',
  'razmer_ustavnogo_kapital_jul',
  'roa',
  'ros',
  'summa_naloga',
  'tax_profit',
  'vyruchka_kompanii_mln_rub_'])

In [37]:
print('Cat features: ')
print()
cat_cols_after_corr = list(set(cat_cols_final)-set(corr_cols_to_drop))
len(cat_cols_after_corr), sorted(cat_cols_after_corr)

Cat features: 



(24,
 ['_massovyj_kontaktnyj_telefon_v_zajavke_ukazan_telefon_na_kotoryj_zaregistrirovano_bolee_h_kompanij_krome_kompanij_agentov_',
  '_organizatsija_pervoe_litso_uchrediteli_vhodit_v_spisok_diskvalifitsirovannyh_',
  '_v_uchrediteljah_uchastnikah_aktsionerah_klienta_uchastie_gosudarstva_bolee_',
  'chislennost_kompanii',
  'data_registratsii',
  'indeks_dolzhnoj_osmotritel_nosti',
  'indeks_finansovogo_riska',
  'kmeans_10',
  'kmeans_2',
  'kmeans_5',
  'kmeans_9',
  'kod_osnovnogo_okved',
  'kompanija_vhodit_v_juridicheskie_litsa_v_sostav_ispolnitel_nyh_organov_kotoryh_vhodjat_diskvalifitsirovannye_litsa',
  'kredity',
  'more_one_close',
  'more_one_close_m',
  'more_one_close_q',
  'more_one_open',
  'more_one_open_m',
  'more_one_open_q',
  'more_one_req_q',
  'opf_organizatsii',
  'otchetnyj_period_god_',
  'sub_ekt_mestonahozhdenija'])

In [38]:
# Сохраним фичи
features_to_save = {'features':features_after_corr,
                   'cont_features':cont_cols_after_corr,
                   'cat_features':cat_cols_after_corr}

with open('features_02.yaml', 'w') as f:
    yaml.dump(features_to_save, f)

### 1. Разделение на train / test. Проверка на лики и неожиданные ошибки в данных по таргету 

In [39]:
# Разделение выборки на трейн и тест с группировкой по ИНН и стратификации по таргету (сохранением средним таргетом)

folds = list(StratifiedGroupKFold(n_splits=5,shuffle=True,random_state=42).split(df,df.target,groups=df.inn))
df_train = df.iloc[folds[4][0]]
print('Train: ', df_train.shape, ';','Target rate: ',  df_train.target.mean())
df_test = df.iloc[folds[4][1]]
print('Test: ', df_test.shape, ';','Target rate: ', df_test.target.mean())

Train:  (52049, 83) ; Target rate:  0.11297047013391227
Test:  (13012, 83) ; Target rate:  0.1152782047340916


In [40]:
# Проверка количества наблюдений и уникальных ИНН при условии, что таргет == 1, 
# для предотвращения попадания одно и того же ИНН в трейн с 1

len(df_train[df_train['target']==1]),len(df_train[df_train['target']==1].inn.unique())

(5880, 5880)

In [41]:
# Проверка таких ИНН в трейне

inn_train_check = pd.DataFrame(df_train[df_train.target==1].groupby('inn')['idinquiry'].count())
inn_train_check = inn_train_check[inn_train_check.idinquiry>1].index.tolist()
df_train[df_train.inn.isin(inn_train_check)].sort_values(by=['inn','data_otkrytija']).head()

,idinquiry,time_zajavki,tip_organizatsii,data_zajavki,year_mon_zajavki,quarter_zajavki,year_zajavki,inn,time_zagruzki_zajavki,kolichestvo_kompanij_s_analogichnym_direktorom_v_tom_zhe_regione,...,tax_profit,kmeans_2,kmeans_3,kmeans_4,kmeans_5,kmeans_6,kmeans_7,kmeans_8,kmeans_9,kmeans_10


In [42]:
# Проверка количества наблюдений и уникальных ИНН при условии, что таргет == 1, 
# для предотвращения попадания одно и того же ИНН в тесте с 1

len(df_test[df_test['target']==1].inn), len(df_test[df_test['target']==1].inn.unique())

(1500, 1500)

In [43]:
# Проверка таких ИНН в тесте

inn_test_check = pd.DataFrame(df_test[df_test.target==1].groupby('inn')['idinquiry'].count())
inn_test_check = inn_test_check[inn_test_check.idinquiry>1].index.tolist()
df_test[df_test.inn.isin(inn_test_check)].sort_values(by=['inn','data_otkrytija']).head()

,idinquiry,time_zajavki,tip_organizatsii,data_zajavki,year_mon_zajavki,quarter_zajavki,year_zajavki,inn,time_zagruzki_zajavki,kolichestvo_kompanij_s_analogichnym_direktorom_v_tom_zhe_regione,...,tax_profit,kmeans_2,kmeans_3,kmeans_4,kmeans_5,kmeans_6,kmeans_7,kmeans_8,kmeans_9,kmeans_10


In [44]:
# Проверка пересечений ИНН в трейне и тесте с таргетом, равным 1, 
# для предотвращения лика в данных (переобучении и заглядывание в будущее)

len(set(df_train[df_train.target==1].inn.unique())&set(df_test[df_test.target==1].inn.unique()))

0

In [45]:
df.to_csv('df.csv',index=False)
df_train.to_csv('df_train.csv',index=False)
df_test.to_csv('df_test.csv',index=False)

In [46]:
print(df.shape)
print(df_train.shape)
print(df_test.shape)
print(len(features_to_save_before['features_before_corr']), len(features_to_save['features']))

(65061, 83)
(52049, 83)
(13012, 83)
57 40
